In [ ]:
!pip install scikeras scikit-learn==1.3.2


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier



In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype("float32") / 255
x_test = x_test.reshape(-1, 28*28).astype("float32") / 255


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def create_model(learning_rate=0.01, hidden_units=64, optimizer='adam'):
    model = keras.Sequential()
    model.add(layers.Dense(hidden_units, activation='relu', input_shape=(784,)))
    model.add(layers.Dense(hidden_units, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    if optimizer == 'sgd':
        opt = optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    elif optimizer == 'rmsprop':
        opt = optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=opt,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
manual_model = create_model(learning_rate=0.01, hidden_units=128, optimizer='sgd')
history = manual_model.fit(x_train, y_train,
                           validation_split=0.2,
                           epochs=5, batch_size=64, verbose=1)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7942 - loss: 0.7217 - val_accuracy: 0.9364 - val_loss: 0.2145
Epoch 2/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9448 - loss: 0.1929 - val_accuracy: 0.9556 - val_loss: 0.1506
Epoch 3/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9629 - loss: 0.1300 - val_accuracy: 0.9637 - val_loss: 0.1218
Epoch 4/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9705 - loss: 0.1015 - val_accuracy: 0.9657 - val_loss: 0.1128
Epoch 5/5
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9776 - loss: 0.0783 - val_accuracy: 0.9688 - val_loss: 0.1047


In [ ]:
param_sets = [
    {"learning_rate": 0.01, "hidden_units": 128, "optimizer": "sgd", "batch_size": 64, "epochs": 5},
    {"learning_rate": 0.001,"hidden_units": 128, "optimizer": "adam","batch_size": 64, "epochs": 5},
    {"learning_rate": 0.01, "hidden_units": 64,  "optimizer": "adam","batch_size": 32, "epochs": 5},
    {"learning_rate": 0.001,"hidden_units": 256, "optimizer": "rmsprop","batch_size": 128,"epochs": 5}
]

In [ ]:
results = []

for params in param_sets:
    print(f"\n🔹 Training with: {params}")
    model = create_model(learning_rate=params["learning_rate"],
                         hidden_units=params["hidden_units"],
                         optimizer=params["optimizer"])

    history = model.fit(x_train, y_train,
                        validation_split=0.2,
                        epochs=params["epochs"],
                        batch_size=params["batch_size"],
                        verbose=0)

    val_acc = history.history["val_accuracy"][-1]
    results.append((val_acc, params))
    print(f"Validation Accuracy: {val_acc:.4f}")


🔹 Training with: {'learning_rate': 0.01, 'hidden_units': 128, 'optimizer': 'sgd', 'batch_size': 64, 'epochs': 5}
Validation Accuracy: 0.9711

🔹 Training with: {'learning_rate': 0.001, 'hidden_units': 128, 'optimizer': 'adam', 'batch_size': 64, 'epochs': 5}
Validation Accuracy: 0.9755

🔹 Training with: {'learning_rate': 0.01, 'hidden_units': 64, 'optimizer': 'adam', 'batch_size': 32, 'epochs': 5}
Validation Accuracy: 0.9560

🔹 Training with: {'learning_rate': 0.001, 'hidden_units': 256, 'optimizer': 'rmsprop', 'batch_size': 128, 'epochs': 5}
Validation Accuracy: 0.9753


In [ ]:
best = max(results, key=lambda x: x[0])
print("\n✅ Best Manual Hyperparameters:")
print(f"Validation Accuracy: {best[0]:.4f}")
print(f"Params: {best[1]}")


✅ Best Manual Hyperparameters:
Validation Accuracy: 0.9755
Params: {'learning_rate': 0.001, 'hidden_units': 128, 'optimizer': 'adam', 'batch_size': 64, 'epochs': 5}


In [ ]:
y_train = y_train.ravel()
model = KerasClassifier(model=create_model, verbose=0)

param_grid = {
    "model__learning_rate": [0.01, 0.001],
    "model__hidden_units": [64, 128],
    "model__optimizer": ['adam', 'sgd'],
    "batch_size": [32, 64],
    "epochs": [3, 5]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, y_train)

print("✅ Best parameters from Grid Search:", grid_result.best_params_)

In [ ]:
param_dist = {
    "model__learning_rate": [0.1, 0.01, 0.001],
    "model__hidden_units": [32, 64, 128, 256],
    "model__optimizer": ['adam', 'sgd', 'rmsprop'],
    "batch_size": [32, 64, 128],
    "epochs": [3, 5, 10]
}

random = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                            n_iter=5, cv=3, n_jobs=-1, random_state=42)
random_result = random.fit(x_train, y_train)

print("✅ Best parameters from Random Search:", random_result.best_params_)